# 파이썬으로 데이터 수집하기
데이터를 수집하고, 전처리하고, 변환하는 데 시간을 많이 보낸다. 이 장에서는 다양한 방식을 통해 데이터를 수집하고, 올바른 포멧으로 파이썬에 전달하는 방법을 배워보자.

## 9.1 stdin과 stdout
커맨드라인으로 파이썬 코드를 실행시킨다면 sys.stdin과 sys.stdout으로 데이터를 파이핑(piping)할 수 있다. \
예를 들어 문서를 읽고 주어진 정규표현식(regex)과 매칭되는 줄을 출력해 주는 코드가 있다고 해보자.

In [1]:
# egrep.py
import sys, re
# sys.argv는 커맨드라인에서 사용할 수 있느 모든 인자에 대한 리스트이다.
# sys.argv[0]는 프로그램의 이름을 나타낸다.
# sys.argv[1]는 커맨드라인에서 주어지는 정규표현식이다.
regex = sys.argv[1]

for line in sys.stdin:
    # regex에 매칭된다면 stdout으로 출력한다.
    if re.search(regex, line):
        sys.stdout.write(line)

In [2]:
# 줄의 개수를 세고 출력해 주는 코드를 작성해 보자
# line_count.py
import sys

count = 0
for line in sys.stdin:
    count += 1

# 출력값은 sys.stdout으로 보낸다.
print(count)

0


In [3]:
# p .122장 데이터 확인, 하고 코드 추가

## 9.2 파일 읽기
파일은 코드에서 바로 읽고 쓸 수 있다.

## 9.2.1 텍스트 파일의 기본
텍스트 파일을 작업하기 위해서는 가장 먼저 open으로 파일 객체를 불러와야 한다.

In [4]:
# 'r'은 read-only(읽기 전용)을 의미한다. 비워 둔다면 읽기 전용으로 가정한다.
file_for_reading1 = open('reading_file.txt', 'r')
file_for_reading2 = open('reading_file.txt')

FileNotFoundError: [Errno 2] No such file or directory: 'reading_file.txt'

In [ ]:
# 'w'는 wrtie(쓰기)를 의미한다.(해당 파일이 이미 존재한다면 기존 파일을 제거한다.)
file_for_writing = open('writing_file.txt', 'w')

In [5]:
# 'a'는 append(덧붙이기)를 의미한다(파일의 맨 끝에 덧붙인다.)
file_for_appending = open('appending_file.txt', 'a')

In [6]:
# 작업이 끝났다면 파일을 닫는 것을 잊지 말자.
file_for_writing.close()

NameError: name 'file_for_writing' is not defined

파일을 닫는 것을 잊지 쉽기 때문에, 항상 작업이 끝나면 파일을 저절로 닫아주는 with안에서 파일 객체를 불러오자

In [ ]:
with open(filename) as f:
    data = function_that_gets_data_from(f)


In [7]:
# 이 시점부터는 f가 이미 종료되었기 때문에 f를 다시 사용하지 말자.
process(data)

NameError: name 'process' is not defined

만약 덱스트 파일 전체가 필요하다면 for를 사용해서 파일의 모든 줄을 반복해서 불러올 수 있다.

In [ ]:
starts_with_hash = 0

with open('input.txt') as f:
    for line in f:  # 파일의 각 줄을 살펴본다.
        if re.match("^#", line):  # regex를 사용해서 줄이 "#"로 시작하는지 확인
            starts_with_hash += 1  # "#"로 시작한다면 1을 추가
            

예를 들어 각 줄마다 이메일 주소 하나가 적혀 있는 파일을 사용해서 메일 도메인에 대한 히스토그램을 그려야 한다면, 도메인 이름은 꽤 정교한 규칙을 다르지만, @뒤에 오는 부분을 도메인으로 보는것은 꽤 괜찮은 결과를 준다.

In [8]:
def get_domain(email_address: str) -> str:
    """'@'기준으로 주소를 자르고 마지막 부분을 반환"""
    return email_address.lower().split("@")[-1]

# 몇 가지 테스트
assert get_domain('joelgrus@gmail.com') == "gmail.com"
assert get_domain('goel@m.datasciencester.com') == 'm.datasciencester.com'

In [9]:
from collections import Counter

with open('email_address.txt', 'r') as f:
    domain_counts = Counter(get_domain(line.strip()) for line in f if "@" in line)

FileNotFoundError: [Errno 2] No such file or directory: 'email_address.txt'

## 9.2.2 구분자가 있는 파일
대부분의 경우 데이터가 들어 있는 파일을 사용하게 될 것이다. 이 파일들은 보통 쉽표(,)나 탭(tab)으로 데이터의 시작과 끝이 구분되어 있다. \
하지만 쉼표, 탭, 개행 문자가 데이터 필드 자체에 포함되어 있으면 데이터 필드를 분리하는 것보다 파이썬에서 제공하는 csv 모듈이나 pandas 라이브러리를 사용하는 것을 추천 \
만약, 파일에 헤더가 없다면 csv.reader로 각 행을 리스트로 바꿀 수 있다. \
예를 들어 탭으로 분리된 주가 파일이 있다면 아래의 코드와 같이 처리 할 수 있다. (주가 파일은 날짜, 종목코드, 주가로 구성, tab으로 각 열 구분)

In [10]:
import csv

with open('tab_delimited_stock_prices.txt') as f:
    tab_reader = csv.reader(f, delimiter ='\t')
    for row in tab_reader:
        date = row[0]
        symbol = row[1]
        closing_price = float(row[2])
        process(date, symbol, closing_price)

FileNotFoundError: [Errno 2] No such file or directory: 'tab_delimited_stock_prices.txt'

만약 파일에 다음과 같이 헤더가 포함되어 있다면 date:symbol:closing_price \
첫 줄에서 reader.next를 사용해서 헤더 행을 건너뛰거나 csv.DictReader를 통해 헤더를 key로 사용하는 딕셔너리로 저장할 수 있다.

In [11]:
with open('colon_delimited_stock_prices.txt') as f:
    colon_reader = csv.DictReader(f, delimiter=":")
    for dict_row in colon_reader:
        date = dict_row["date"]
        symbol = dict_row["symbol"]
        closing_price = float(dict_row["closing_price"])
        process(date, symbol, closing_price)

FileNotFoundError: [Errno 2] No such file or directory: 'colon_delimited_stock_prices.txt'

파일의 헤더가 없더라도 filenames 파라미터로 key를 설정해서 DictReader를 사용할 수 있다. \
csv.writer를 사용해서 구분자가 있는 파일을 생성할 수도 있다.

In [12]:
todays_prices = {'AAPL': 90.91, 'MSFT': 41.68, 'FB': 64.5}

with open('comma_delimited_stock_prices.txt', 'w') as f:
    csv_writer = csv.writer(f, delimiter=',')
    for stock, price in todays_prices.items():
        csv_writer.writerow([stock, price])

데이터 필드 자체에 쉼표가 포함되어 있더라도 csv.writer는 올바른 방식으로 처리해 줄 것이다. 하지만 구분자가 있는 파일을 생성해 주는 코드를 직접 \
만들어 사용한다면 문제가 발생할 수도 있다. 예를 들어 아래와 같은 코드는

In [13]:
results = [["test1", "success", "Monday"],
         ["test2", "success, kind of", "Tuesday"],
         ["test3", "failure, kind of", "Wednesday"],
         ["test4", "failure, utter", "Thursday"]]

# 이렇게 하면 안된다.
with open('bad_csv.txt', 'w') as f:
    for row in results:
        f.write(",".join(map(str, row)))  # 필드 안에 쉼표가 있을 수도 있다!
        f.write("\n")  # 필드 안에 새로운 줄이 있을 수도 있다.

결과는 아무도 알아볼 수 없는 .csv 파일이 만들어질 것이다.

In [14]:
with open('bad_csv.txt', 'r') as f:
    lines = f.readlines()
    for line in lines:
        print(line)

test1,success,Monday

test2,success, kind of,Tuesday

test3,failure, kind of,Wednesday

test4,failure, utter,Thursday



## 9.3 웹 스크래핑
웹 스크래핑(웹 페이지에서 데이터를 긁어오는 것)을 통해서 데이터를 얻을 수 있다. 웹페이지를 갖고 오는 것은 놀랍게도 상당히 쉬운 작업이다. 하지만 의미 있고 구조화된 정보를 갖고 오는 것은 그렇게 쉬운 일이 아니다.

## 9.3.1 HTML과 파싱
웹페이지는 HTML로 작성되었고, HTML의 내용은 요소(element)와 속성(attribute)로 구성되어 있다. \
실제로는 제대로 된 형태를 갖추거나 주석이 달린 HTML을 찾기 힘들기 때문에 HTML을 이해하는 데 도움을 줄 수 있는 도구가 필요하다. \
HTML에서 데이터를 추출하기 위해 웹페이지의 HTML 요소를 나무(tree) 구조로 저장해서 쉽게 접근하게 해주는 Beautiful Soup 라이브러리를 사용할 것이다. 효과적으로 HTTP 요청을 할 수 있는 Requests 라이브러리를 설치하자. \
Python에 기본적으로 탑제된 HTML parser(파서)는 완벽한 HTML구조가 아니면 제대로 작동하지 않기 때문에 html5lib 파서를 설치해서 사용할 것\

In [15]:
from bs4 import BeautifulSoup
import requests

# 사용할 HTML은 깃허브에 있다.
# 공백으로 분리된 문자열은 하나로 연결다는 것을 기억!

url = ("https://raw.githubusercontent.com/"
       "joelgrus/data/master/getting-data.html")
html = requests.get(url).text
soup = BeautifulSoup(html, 'html5lib')

첫 번째 p 태그와 해당 태그 속에 포함된 정보를 찾고 싶다면 다음과 같은 코드를 사용할 수 있다.

In [16]:
first_paragraph = soup.find('p')  # 혹은 soup.p
print(first_paragraph)

<p id="p1">This is the first paragraph.</p>


그리고 text 속성을 이용하면 Tag의 텍스트 내용을 추출할 수 있다.

In [17]:
first_paragraph_text = soup.p.text
first_paragraph_words = soup.p.text.split()
print("first_paragraph_text:", first_paragraph_text)
print("first_paragraph_words:", first_paragraph_words)

first_paragraph_text: This is the first paragraph.
first_paragraph_words: ['This', 'is', 'the', 'first', 'paragraph.']


또한 태그를 딕셔너리인 것처럼 사용함으로써 태그의 속성을 추출할 수 있다.

In [18]:
first_paragraph_id = soup.p['id']  # id가 존재하지 않으면 KeyError 발생
first_paragraph_id2 = soup.p.get('id')  # id가 없다면 None을 반환
print(first_paragraph_id, first_paragraph_id2)

p1 p1


여러 태그를 한 번에 불러올 수도 있다.

In [19]:
all_paragraphs = soup.find_all("p")  # 혹은 just soup('p')
paragraphs_with_ids = [p for p in soup('p') if p.get("id")]
print(all_paragraphs)
print(paragraphs_with_ids)

[<p id="p1">This is the first paragraph.</p>, <p class="important">This is the second paragraph.</p>]
[<p id="p1">This is the first paragraph.</p>]


특정 class의 태그가 필요한 경우

In [20]:
important_paragraphs = soup('p', {'class': 'important'})
important_paragraphs2 = soup('p', 'important')
important_paragraphs3 = [p for p in soup('p') if 'important' in p.get('class', [])]
print(important_paragraphs)
print(important_paragraphs2)
print(important_paragraphs3)

[<p class="important">This is the second paragraph.</p>]
[<p class="important">This is the second paragraph.</p>]
[<p class="important">This is the second paragraph.</p>]


In [21]:
# span 요소 안에 포함된 모든 div요소 찾기
# 주의: 만약 여러 div안에 똑같은 span이 존재한다면, 동일한 span을 중복으로 반환, 이런 경우 더욱 정교한 논리가 필요
span_inside_divs = [span 
                    for div in soup('div')  # 모든 <div>에
                    for span in div('span')]  # 포함된 <span>을 탐색
print(span_inside_divs)

[<span id="name">Joel</span>, <span id="twitter">@joelgrus</span>, <span id="email">joelgrus-at-gmail</span>]


## 9.3.2 예시: 의회 감시하기

In [22]:
from bs4 import BeautifulSoup
import requests

url = "https://www.house.gov/representatives"
text = requests.get(url).text
soup = BeautifulSoup(text, "html5lib")

all_urls = [a['href'] 
            for a in soup('a')
           if a.has_attr('href')]

print(len(all_urls))  # 너무 많다.

966


In [23]:
import re
# 정규식을 활용, 시작이 http:// or https:// 이고 끝이 .house.gov or .house.gov/
# http:// or https:// 시작하고 .house.gov or .house.gov//로 끝나야 한다.

regex = r"^https?://.*\.house\.gov/?$"

In [24]:
# 테스트
assert re.match(regex, "http://joel.house.gov")
assert re.match(regex, "https://joel.house.gov")
assert re.match(regex, "http://joel.house.gov/")
assert re.match(regex, "https://joel.house.gov/")
assert not re.match(regex, "joel.house.gov")
assert not re.match(regex, "https://joel.house.com")
assert not re.match(regex, "https://joel.house.gov/biography")

In [25]:
# 적용하기
good_urls = [url for url in all_urls if re.match(regex, url)]
print(len(good_urls))

872


In [26]:
# 중복 제거
good_urls = list(set(good_urls))
print(len(good_urls))

436


In [30]:
html = requests.get('https://jayapal.house.gov').text
soup = BeautifulSoup(html, 'html5lib')

In [31]:
# 링크가 여러 개 있을 수도 있으니 set을 사용해 중복 제거
links = {a['href'] for a in soup('a') if 'press releases' in a.text.lower()}
print(links)

{'https://jayapal.house.gov/category/press-releases/', 'https://jayapal.house.gov/category/news/'}


In [32]:
# 웹 스크래핑
from typing import Dict, Set

press_releases: Dict[str, Set[str]] = {}

for house_url in good_urls:
    html = requests.get(house_url).text
    soup = BeautifulSoup(html, 'html5lib')
    pr_links = {a['href'] for a in soup('a') if 'press releases' in a.text.lower()}

print(f"{house_url}: {pr_links}")
press_releases[house_url] = pr_links

https://ebjohnson.house.gov: {'/media-center/press-releases'}


멋대로 사이트를 스크래핑하는 것은 무례한 행동이다. 대부분의 사이트는 얼마나 자주 스크래핑 해도 좋은지 표시하는 robots.txt파일이 있다.

In [33]:
# 페이지에서 특정 단어를 포함하고 있는지 확인해 주는 일반적인 함수
def paragraph_mentions(text: str, keyword: str) -> bool:
    """
    텍스트 안의 <p>가 특정 {keyword}를 언급하면 True를 반환한다.
    """
    soup = BeautifulSoup(text, 'html5lib')
    paragraphs = [p.get_text() for p in soup('p')]
    
    return any(keyword.lower() in paragraph.lower()
               for paragraph in paragraphs)

In [34]:
# 테스트
text = """<body><h1>Facebook</h1><p>Twitter</p>"""
assert paragraph_mentions(text, "Twitter")  # <p> 안에 있는 경우
assert not paragraph_mentions(text, "Facebook")  # <p> 안에 있지 않은 경우

In [35]:
for house_url, pr_links in press_releases.items():
    for pr_link in pr_links:
        url = f"{house_url}/{pr_link}"
        text = requests.get(url).text
        
        if paragraph_mentions(text, "data"):
            print(f"{house_url}")
            break

## 9.4 API 사용하기
많은 웹사이트는 API(Application Programming Interface)를 통해 사이트의 데이터를 구조화된 형태로 제공해주고 있다. API를 사용하면 직접 스크래핑하 필요가 없다.

## 9.4.1 JSON과 XML
텍스트 교환을 위해 정보 통신 규약인 HTTP에 따라 API를 통해 요청하는 데이터는 **직렬화**된 문자열 형태로 유지되어 있어야 한다. 대부분의 경우, JSON(JavaScript Object Notation)을 사용한다. JSON은 Python dict와 비슷하다.

In [36]:
{"title": "Data Science Book",
"author": "Joel Grus",
"publicationYear": 2019,
"topics": ["data", "science", "data science"]}

{'title': 'Data Science Book',
 'author': 'Joel Grus',
 'publicationYear': 2019,
 'topics': ['data', 'science', 'data science']}

Python의 json 모듈을 통해 파싱이 JSON 파싱이 가능하다. 모듈의 loads 함수를 사용하면 직렬화된 문자열 형태의 JSON 객체를 파이썬 객체로 분리시킬 수 있다.

In [37]:
import json
serialized = """{"title": "Data Science Book",
"author": "Joel Grus",
"publicationYear": 2019,
"topics": ["data", "science", "data science"]}"""

# JSON을 Python dict로 파싱
deserialized = json.loads(serialized)
assert deserialized["publicationYear"] == 2019
assert "data science" in deserialized["topics"]

## 9.4.2 인증이 필요하지 않은 API 사용하기
대부분의 API를 사용하기 위해서는 사용자 인증을 거쳐야 한다. 이것은 복잡한 작업을 요구한다. 사용자 인증없이 간단한 작업이 가능한 깃허브 API가 있다.

In [38]:
import requests, json

github_user = "joelgrus"
endpoint = f"https://api.github.com/users/{github_user}/repos"

repos = json.loads(requests.get(endpoint).text)

사용자가 어떤 월이나 어떤 요일에 가장 자주 저장소를 만드는지 확인이 가능하다. 날짜가 유니코드로 되어있으니 python dateutil을 사용하여 파싱 가능

In [39]:
from collections import Counter
from dateutil.parser import parse
import dateutil

dates = [parse(repo["created_at"]) for repo in repos]
month_counts = Counter(date.month for data in dates)
weekday_counts = Counter(date.weekday() for date in dates)

NameError: name 'date' is not defined

또한 가장 최근에 만든 저장소 5개에서 사용된 프로그래밍 언어를 다음과 같이 확인해 볼 수 있다.

In [40]:
last_5_repositories = sorted(repos,
                            key=lambda r: r["pushed_at"],
                            reverse=True)[:5]

last_5_languages = [repo["languages"] for repo in last_5_repositories]

KeyError: 'languages'

보통은 API로 요청한 데이터를 직접 파싱할 일이 거의 없다. Python의 장점 중 하나는 이미 누군가가 세상에 거의 모든 API에 접근할 수 있는 라이브러리를 만들어 놨다는 것이다. 잘 만들어진 라이브러리라면 골치 아픈 사용자 인증 절차 없이도 API를 손쉽게 사용할 수 있다. 하지만 그렇지 안거나 오래된 API 기반으로 만들어진 라이브러리라면 골치 아픈 일이 많을 것이다. 가끔은 직접 API 접근용 라이브러리를 만들어야 할 수도 있어서 기본적인 내용은 알고 있는게 좋다.

### 9.4.3 API 찾기
특성 사이트의 데이터가 필요하다면 사이트의 developers 혹은 API라는 페이지에서 관련 내용을 찾아보거나, 인터넷에서 python 무슨 무슨 api라고 검색하여 필요한 라이브러리를 찾아볼 수 있다. 원하는 API를 찾을 수 없다면 최후의 보루인 스크래핑이 있다는 것을 기억하자

## 9.5 예시: 트위터 API 사용하기
트위터에는 다양하게 적용해 볼 수 있는 엄청난 양의 데이터가 존재한다. 실시간 뉴스나 시사 문제에 관한 반응을 살펴보거나 특정 주제와 관련된 링크 또한 찾아볼 수 있다. 데이터에 접근만 할 수 있다면 많은 것을 해볼 수 있다. 트위터 API를 사용하면 데이터에 접근할 수 있다. 트위터 API를 사용하기 위해 Twython 라이브러리를 이용해보자 (pip install twyhon)

## 9.5.1 인증받기
트위터 API를 사용하기 위해서는 개인 트위터 계정을 통해 인증을 받아야한다. 

### Twython 사용하기
트위터 API를 사용하는데 가장 까다로운 부분은 본인 인증을 하는 부분이다. API 제공자는 데이터를 접근하기 위한 인증을 받았는지, 사용량 제한을 넘지는 않았는지, 누가 데이터에 접근하는지도 알고 싶어한다. 

In [42]:
# 코드 더 필요 현재 트위터 계정이 없음